### Agent Tool + Save searched information to vector db 

In [112]:
# API 키를 환경변수로 관리하기 위한 설정 파일
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.tools import Tool

In [113]:
# API 키 정보 로드
load_dotenv()
upstage_api_key = os.getenv("UPSTAGE_API_KEY")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=25)

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("./SPRI_AI_Brief_2023년12월호_F.pdf")

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

In [114]:
# VectorStore를 생성합니다.
embeddings = UpstageEmbeddings(api_key=upstage_api_key, model="embedding-query")
vector = FAISS.from_documents(split_docs, embeddings)

vector.save_local("faiss_index_agent_rerank")

# Retriever를 생성합니다.
retriever = vector.as_retriever(search_kwargs={"k": 5})

In [ ]:
# Vector DB 로컬에 저장 후 로드 하는 예제 코드
# # 파일 경로
# index_path = "faiss_index_rerank"

# # 벡터 스토어 생성 또는 로드
# if os.path.exists(index_path):
#     print("기존 FAISS 인덱스를 로드합니다.")
#     vector = FAISS.load_local(
#         index_path, embeddings, allow_dangerous_deserialization=True
#     )
#     vector.add_documents(splits)
# else:
#     print("새로운 FAISS 인덱스를 생성합니다.")
#     vector = FAISS.from_documents(splits, embeddings)

# # 인덱스 저장
# vector.save_local(index_path)
# retriever = vector.as_retriever(search_kwargs={"k": 5})

In [115]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(
    max_results=3, include_domains=["naver.com", "google.com", "sports.news.naver.com"]
)

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

In [116]:
class SearchAndStoreTool:
    def __init__(self, search_tool, vectorstore, embedding_model, text_splitter):
        self.search_tool = search_tool
        self.vectorstore = vectorstore
        self.embedding_model = embedding_model
        self.text_splitter = text_splitter

    def run(self, query: str) -> str:
        # 1. 검색
        search_results = self.search_tool.invoke(query)

        # 2. Document 형식으로 변환
        docs = [
            Document(
                page_content=item["content"],
                metadata={
                    "title": item.get("title", ""),
                    "url": item.get("url", ""),
                    "score": item.get("score", 0.0),
                },
            )
            for item in search_results
        ]
        splits = text_splitter.split_documents(docs)
        # 3. 벡터화 및 저장
        self.vectorstore.add_documents(splits)
        self.vectorstore.save_local("faiss_index_agent_rerank")

        return f"Stored {len(splits)} documents from web search."

In [117]:
# 도구로 wrapping
search_tool = Tool(
    name="search_tool",
    func=SearchAndStoreTool(search, vector, embeddings, text_splitter).run,
    description="Search information from web and store the results to the vector database.",
)

### rerank 추가

In [118]:
from sentence_transformers import CrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
import torch

# model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
# reranker = CrossEncoderReranker(model=model, top_n=3)
reranker = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    device="cuda" if torch.cuda.is_available() else "cpu",
)

In [119]:
class RerankTool:
    def __init__(self, reranker, retriever):
        self.reranker = reranker
        self.retriever = retriever

    def __rerank__(self, query: str, top_k=3):
        # docs = self.retriever.get_relevant_documents(query)
        docs = self.retriever.invoke(query)
        # 각 문서와 쿼리를 쌍으로 만들어 점수 계산
        pairs = [(query, doc.page_content) for doc in docs]
        scores = self.reranker.predict(pairs)

        # 점수와 함께 문서를 묶어서 정렬
        scored_docs = list(zip(scores, docs))
        scored_docs.sort(key=lambda x: x[0], reverse=True)

        # top_k 문서만 선택
        return [doc for _, doc in scored_docs[:top_k]]

    def output(self, query: str):
        top_docs = self.__rerank__(query)
        context = "\n\n".join([doc.page_content for doc in top_docs])

        return context

In [120]:
# 도구로 wrapping
rerank_tool = Tool(
    name="rerank_tool",
    func=RerankTool(reranker, retriever).output,
    description="Search reranked information from vector database for final answer.",
)

In [121]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
# tools = [search, retriever_tool]
tools = [search_tool, retriever_tool, rerank_tool]

In [122]:
from langchain_openai import ChatOpenAI
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatUpstage(model_name="solar-mini")

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "First, make sure to use the `pdf_search` tool for searching information from the PDF document."
            "If you can't find the information from the PDF document, you MUST use `search_tool` tool for searching information from the web."
            "Never answer a question based on your own knowledge without using tools."
            "Finally use `rerank_tool` to get information, before answering the question"
            "After all, you must return answer about question.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [ ]:
# test_query = "삼성전자의 생성형 AI 전략"
# context = rerank_tool(test_query)

# print("=== Reranked Context ===")
# print(context)

C:\Users\ediso\AppData\Local\Temp\ipykernel_14048\2276822396.py:2: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = rerank_tool(test_query)
C:\Users\ediso\AppData\Local\Temp\ipykernel_14048\4225406442.py:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = self.retriever.get_relevant_documents(query)


=== Reranked Context ===
저해상도 이미지의 고해상도 전환도 지원
nIT 전문지 테크리퍼블릭 (TechRepublic) 은 온디바이스 AI가 주요 기술 트렌드로 부상했다며 , 
2024 년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Llama)2 를 탑재한 퀄컴 기기 및 구글 
어시스턴트를 적용한 구글 픽셀(Pixel) 과 경쟁할 것으로 예상
☞ 출처 : 삼성전자 , ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스 ’ 공개, 2023.11.08.
삼성전자 , ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.

SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 

In [123]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [124]:
# 질의에 대한 답변을 출력

result = agent_executor.invoke(
    {"input": "2024년 한국 프로야구 플레이오프 진출한 4팀을 검색하여 알려주세요."}
)
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': '2024년 한국 프로야구 플레이오프 진출한 4팀'}`


행사명 행사 주요 개요
CES 2024
-미국 소비자기술 협회(CTA) 가 주관하는 세계 최대 가전·IT·소
비재 전시회로 5G, AR&VR, 디지털헬스 , 교통·모빌리티 등 
주요 카테고리 중심으로 기업들이 최신의 기술 제품군을 전시
-CTA 사피로 회장은 가장 주목받는 섹터로 AI를 조명하였으며 , 
모든 산업을 포괄한다는 의미에서 ‘올 온(All on)’을 주제로 한 
이번 전시에는 500곳 이상의 한국기업 참가 예정
기간 장소 홈페이지
2024.1.9~12 미국, 라스베가스 https://www.ces.tech/
AIMLA 2024
-머신러닝 및 응용에 관한 국제 컨퍼런스 (AIMLA 2024) 는 
인공지능 및 머신러닝의 이론, 방법론 및 실용적 접근에 관한 
지식과  최신 연구 결과 공유
-이론 및 실무 측면에서 인공지능 , 기계학습의 주요 분야를 
논의하고 , 학계, 산업계의 연구자와 실무자들에게 해당 분
야의 최첨단 개발 소식 공유
기간 장소 홈페이지

∙한편, 2023 년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해 
4년간 30억 달러 규모의 계약을 체결
∙오픈AI 창업자 그룹의 일원이었던 다리오 (Dario Amodei) 와 다니엘라 아모데이 (Daniela Amodei) 
남매가 2021 년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드 (Claude)’ LLM을 개발
n아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인  오픈
AI와 협력을 확대
∙마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023 년 1월 추가로 100억 달러를 
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며 , 오픈AI는 마이크로소프트의 애저(Azure) 
클라우드 플랫폼을 

In [110]:
result = agent_executor.invoke(
    {"input": "네이버가 자체 개발한 생성형 AI에 대해서 알려줘."}
)
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': '네이버가 자체 개발한 생성형 AI'}`


SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 ,

▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6
 
 2. 기업/산업 
   ▹ 미국 프런티어 모델 포럼, 1,000 만 달러 규모의 AI 안전 기금 조성································ 7
   ▹ 코히어 , 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8
   ▹ 알리바바 클라우드 , 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9
   ▹ 삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’